# Task 1: Exploratory Data Analysis and Data Preprocessing
## Intelligent Complaint Analysis for Financial Services

**Objective:** Understand the structure, content, and quality of the complaint data and prepare it for the RAG pipeline.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path

# Set style for visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Load the Dataset

In [ ]:
# Load the full CFPB complaint dataset
data_path = Path('../data/raw/complaints.csv')
print(f"Loading data from: {data_path}")

# Read the CSV file
df = pd.read_csv(data_path, low_memory=False)

print(f"\nDataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

## 2. Initial Data Exploration

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Data types and missing values
print("Data Info:")
df.info()

In [ ]:
# Statistical summary
df.describe(include='all')

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
}).sort_values('Percentage', ascending=False)

print("Missing Values Analysis:")
print(missing_df[missing_df['Missing Count'] > 0])

## 3. Product Distribution Analysis

In [ ]:
# Analyze distribution of complaints across products
print("Product Distribution:")
product_counts = df['Product'].value_counts()
print(product_counts)

# Visualize product distribution
plt.figure(figsize=(14, 6))
product_counts.head(15).plot(kind='barh', color='steelblue')
plt.title('Top 15 Products by Complaint Count', fontsize=16, fontweight='bold')
plt.xlabel('Number of Complaints', fontsize=12)
plt.ylabel('Product', fontsize=12)
plt.tight_layout()
plt.show()

## 4. Narrative Analysis

In [ ]:
# Identify the narrative column (it might be named differently)
narrative_columns = [col for col in df.columns if 'narrative' in col.lower() or 'complaint' in col.lower()]
print(f"Potential narrative columns: {narrative_columns}")

# Assuming the column is 'Consumer complaint narrative'
narrative_col = 'Consumer complaint narrative' if 'Consumer complaint narrative' in df.columns else narrative_columns[0]
print(f"\nUsing column: {narrative_col}")

In [ ]:
# Count complaints with and without narratives
narratives_present = df[narrative_col].notna().sum()
narratives_missing = df[narrative_col].isna().sum()

print(f"Complaints WITH narratives: {narratives_present:,} ({narratives_present/len(df)*100:.2f}%)")
print(f"Complaints WITHOUT narratives: {narratives_missing:,} ({narratives_missing/len(df)*100:.2f}%)")

# Visualize
plt.figure(figsize=(8, 6))
plt.pie([narratives_present, narratives_missing], 
        labels=['With Narrative', 'Without Narrative'],
        autopct='%1.1f%%',
        colors=['#2ecc71', '#e74c3c'],
        startangle=90)
plt.title('Distribution of Complaints by Narrative Availability', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Calculate word count for narratives
df['narrative_word_count'] = df[narrative_col].fillna('').apply(lambda x: len(str(x).split()))

# Filter only rows with narratives for analysis
df_with_narratives = df[df[narrative_col].notna()].copy()

print("Narrative Length Statistics (word count):")
print(df_with_narratives['narrative_word_count'].describe())

In [ ]:
# Visualize narrative length distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(df_with_narratives['narrative_word_count'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Word Count', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Narrative Lengths', fontsize=14, fontweight='bold')
axes[0].axvline(df_with_narratives['narrative_word_count'].median(), color='red', linestyle='--', label='Median')
axes[0].legend()

# Box plot
axes[1].boxplot(df_with_narratives['narrative_word_count'], vert=True)
axes[1].set_ylabel('Word Count', fontsize=12)
axes[1].set_title('Box Plot of Narrative Lengths', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Identify very short and very long narratives
very_short = df_with_narratives[df_with_narratives['narrative_word_count'] < 10]
very_long = df_with_narratives[df_with_narratives['narrative_word_count'] > 500]

print(f"Very short narratives (<10 words): {len(very_short):,}")
print(f"Very long narratives (>500 words): {len(very_long):,}")

# Sample short narrative
if len(very_short) > 0:
    print("\nExample of a very short narrative:")
    print(very_short[narrative_col].iloc[0])

# Sample long narrative
if len(very_long) > 0:
    print("\nExample of a very long narrative (first 500 chars):")
    print(very_long[narrative_col].iloc[0][:500] + "...")

## 5. Filter Dataset for Target Products

In [ ]:
# Define target products
target_products = [
    'Credit card',
    'Credit card or prepaid card',
    'Personal loan',
    'Savings account',
    'Money transfer',
    'Money transfers'
]

# Check exact product names in the dataset
print("All unique products in dataset:")
all_products = df['Product'].unique()
for product in sorted(all_products):
    print(f"  - {product}")

In [ ]:
# Filter for target products (case-insensitive partial matching)
def matches_target_product(product):
    if pd.isna(product):
        return False
    product_lower = str(product).lower()
    return any([
        'credit card' in product_lower,
        'personal loan' in product_lower,
        'savings account' in product_lower,
        'money transfer' in product_lower
    ])

df_filtered = df[df['Product'].apply(matches_target_product)].copy()

print(f"\nOriginal dataset size: {len(df):,}")
print(f"After product filtering: {len(df_filtered):,}")
print(f"Reduction: {len(df) - len(df_filtered):,} rows ({(len(df) - len(df_filtered))/len(df)*100:.2f}%)")

In [ ]:
# Remove records with empty narratives
df_filtered = df_filtered[df_filtered[narrative_col].notna()].copy()

print(f"After removing empty narratives: {len(df_filtered):,}")

# Show distribution of filtered products
print("\nFiltered Product Distribution:")
print(df_filtered['Product'].value_counts())

In [ ]:
# Visualize filtered product distribution
plt.figure(figsize=(10, 6))
df_filtered['Product'].value_counts().plot(kind='bar', color='coral')
plt.title('Complaint Distribution for Target Products', fontsize=16, fontweight='bold')
plt.xlabel('Product', fontsize=12)
plt.ylabel('Number of Complaints', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 6. Text Cleaning and Preprocessing

In [ ]:
def clean_text(text):
    """
    Clean complaint narrative text.
    - Convert to lowercase
    - Remove special characters (keep alphanumeric and basic punctuation)
    - Remove extra whitespace
    - Remove common boilerplate phrases
    """
    if pd.isna(text):
        return ""
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove common boilerplate phrases
    boilerplate_patterns = [
        r'i am writing to file a complaint',
        r'i would like to file a complaint',
        r'this is a complaint about',
        r'xxxx',  # Common redaction marker
    ]
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Keep only alphanumeric characters and basic punctuation
    text = re.sub(r'[^a-z0-9\s.,!?;:\-\'"()]', ' ', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Test the cleaning function
sample_text = df_filtered[narrative_col].iloc[0]
print("Original text (first 300 chars):")
print(sample_text[:300])
print("\nCleaned text (first 300 chars):")
print(clean_text(sample_text)[:300])

In [ ]:
# Apply cleaning to all narratives
print("Cleaning narratives...")
df_filtered['cleaned_narrative'] = df_filtered[narrative_col].apply(clean_text)

# Remove any rows where cleaned narrative is empty
df_filtered = df_filtered[df_filtered['cleaned_narrative'].str.len() > 0].copy()

print(f"Final dataset size after cleaning: {len(df_filtered):,}")

In [ ]:
# Calculate cleaned narrative statistics
df_filtered['cleaned_word_count'] = df_filtered['cleaned_narrative'].apply(lambda x: len(x.split()))

print("Cleaned Narrative Statistics:")
print(df_filtered['cleaned_word_count'].describe())

# Compare before and after cleaning
comparison = pd.DataFrame({
    'Original': df_filtered['narrative_word_count'].describe(),
    'Cleaned': df_filtered['cleaned_word_count'].describe()
})
print("\nComparison of Word Counts:")
print(comparison)

## 7. Additional Analysis

In [ ]:
# Analyze issues and sub-issues if available
if 'Issue' in df_filtered.columns:
    print("Top 10 Issues:")
    print(df_filtered['Issue'].value_counts().head(10))
    
    # Visualize
    plt.figure(figsize=(12, 6))
    df_filtered['Issue'].value_counts().head(10).plot(kind='barh', color='teal')
    plt.title('Top 10 Complaint Issues', fontsize=14, fontweight='bold')
    plt.xlabel('Count', fontsize=12)
    plt.ylabel('Issue', fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
# Temporal analysis if date column exists
date_columns = [col for col in df_filtered.columns if 'date' in col.lower()]
if date_columns:
    date_col = date_columns[0]
    print(f"Using date column: {date_col}")
    
    df_filtered[date_col] = pd.to_datetime(df_filtered[date_col], errors='coerce')
    df_filtered['year'] = df_filtered[date_col].dt.year
    df_filtered['month'] = df_filtered[date_col].dt.month
    
    # Complaints over time
    plt.figure(figsize=(14, 6))
    df_filtered.groupby('year').size().plot(kind='line', marker='o', color='purple', linewidth=2)
    plt.title('Complaints Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of Complaints', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 8. Save Filtered and Cleaned Dataset

In [ ]:
# Select relevant columns for the processed dataset
columns_to_keep = [
    'Product',
    'Issue',
    'Sub-issue',
    'Company',
    'State',
    'Date received',
    narrative_col,
    'cleaned_narrative'
]

# Filter to only existing columns
columns_to_keep = [col for col in columns_to_keep if col in df_filtered.columns]

df_final = df_filtered[columns_to_keep].copy()

# Add a unique complaint ID if not present
if 'Complaint ID' not in df_final.columns:
    df_final.insert(0, 'complaint_id', range(1, len(df_final) + 1))
else:
    df_final.rename(columns={'Complaint ID': 'complaint_id'}, inplace=True)

print(f"Final dataset shape: {df_final.shape}")
print(f"Columns: {df_final.columns.tolist()}")

In [ ]:
# Save to processed data folder
output_path = Path('../data/processed/filtered_complaints.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

df_final.to_csv(output_path, index=False)
print(f"\n✅ Filtered and cleaned dataset saved to: {output_path}")
print(f"Total records: {len(df_final):,}")

## 9. Summary of Findings

### Key Findings from EDA:

1. **Dataset Overview:**
   - Original dataset contained [X] complaints
   - After filtering for target products and removing empty narratives: [Y] complaints

2. **Product Distribution:**
   - Credit cards account for the majority of complaints
   - [Add specific insights from your analysis]

3. **Narrative Characteristics:**
   - Average narrative length: [X] words
   - Median narrative length: [Y] words
   - Range: [min] to [max] words
   - Some narratives are very short (<10 words) and may have limited context
   - Some narratives are very long (>500 words) and will benefit from chunking

4. **Data Quality:**
   - [X]% of complaints had narratives
   - Text cleaning removed boilerplate phrases and special characters
   - All target product categories are well-represented

### Next Steps:
- Proceed to Task 2: Text Chunking and Vector Store Indexing
- Use the cleaned narratives for embedding generation
- Consider chunk size based on narrative length distribution